# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [9]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

data_path = '../data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
df['Embarked'] = df['Embarked'].fillna('None')
df['Age'] = df['Age'].fillna(df['Age'].mean())

mean_df = df.groupby(['Embarked'])['Age'].mean().reset_index()
mode_df = df.groupby(['Embarked'])['Age'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Embarked'])['Age'].median().reset_index()
max_df = df.groupby(['Embarked'])['Age'].max().reset_index()
min_df = df.groupby(['Embarked'])['Age'].min().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Embarked'])
temp = pd.merge(temp, median_df, how='left', on=['Embarked'])
temp = pd.merge(temp, max_df, how='left', on=['Embarked'])
temp = pd.merge(temp, min_df, how='left', on=['Embarked'])
temp.columns = ['Embarked', 'Age_Mean', 'Age_Mode', 'Age_Median', 'Age_Max', 'Age_Min']
temp.head()

,Embarked,Age_Mean,Age_Mode,Age_Median,Age_Max,Age_Min
0,C,30.562419,29.699118,29.699118,71.0,0.42
1,None,50.000000,38.000000,50.000000,62.0,38.00
2,Q,29.113724,29.699118,29.699118,70.5,2.00
3,S,29.480855,29.699118,29.699118,80.0,0.67


In [4]:
df = pd.merge(df, temp, how='left', on=['Embarked'])
df = df.drop(['Embarked'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Age_Mean,Age_Mode,Age_Median,Age_Max,Age_Min
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,29.480855,29.699118,29.699118,80.0,0.67
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,30.562419,29.699118,29.699118,71.0,0.42
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,29.480855,29.699118,29.699118,80.0,0.67
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,29.480855,29.699118,29.699118,80.0,0.67
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,29.480855,29.699118,29.699118,80.0,0.67


In [5]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

10 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Age_Mean', 'Age_Mode', 'Age_Median', 'Age_Max', 'Age_Min']



,Pclass,Age,SibSp,Parch,Fare,Age_Mean,Age_Mode,Age_Median,Age_Max,Age_Min
0,3,22.0,1,0,7.2500,29.480855,29.699118,29.699118,80.0,0.67
1,1,38.0,1,0,71.2833,30.562419,29.699118,29.699118,71.0,0.42
2,3,26.0,0,0,7.9250,29.480855,29.699118,29.699118,80.0,0.67
3,1,35.0,1,0,53.1000,29.480855,29.699118,29.699118,80.0,0.67
4,3,35.0,0,0,8.0500,29.480855,29.699118,29.699118,80.0,0.67


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [6]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
# 沒有這四個新特徵的 dataframe 稱為 df_minus
df_minus = df.drop(['Age_Mean', 'Age_Mode', 'Age_Median', 'Age_Max', 'Age_Min'] , axis=1)

# 原始特徵 + 邏輯斯迴歸
train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7004142866110099

In [7]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
# 新特徵 + 邏輯斯迴歸
train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7138974326784258

In [10]:
# 原始特徵 + 隨機森林
train_X = MMEncoder.fit_transform(df_minus)
estimator = RandomForestRegressor()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.034557262542503485

In [12]:
# 新特徵 + 隨機森林
train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

ValueError: could not convert string to float: 'Braund, Mr. Owen Harris'